In [ ]:
!pip install transformers datasets sentencepiece nltk pandas


In [ ]:
from huggingface_hub import login

login()


In [ ]:
import pandas as pd

df = pd.read_csv("/content/Corona_NLP_test.csv")  # change filename to your file
df.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,1,44953,NYC,02-03-2020,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative
1,2,44954,"Seattle, WA",02-03-2020,When I couldn't find hand sanitizer at Fred Me...,Positive
2,3,44955,NaN,02-03-2020,Find out how you can protect yourself and love...,Extremely Positive
3,4,44956,Chicagoland,02-03-2020,#Panic buying hits #NewYork City as anxious sh...,Negative
4,5,44957,"Melbourne, Victoria",03-03-2020,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def clean_text(text):
    encoded = tokenizer.encode(text, add_special_tokens=False)
    return tokenizer.decode(encoded)

df["clean_text_hf"] = df["OriginalTweet"].astype(str).apply(clean_text)
df.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment,clean_text_hf
0,1,44953,NYC,02-03-2020,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative,trending : new yorkers encounter empty superma...
1,2,44954,"Seattle, WA",02-03-2020,When I couldn't find hand sanitizer at Fred Me...,Positive,when i couldn ' t find hand sanitizer at fred ...
2,3,44955,NaN,02-03-2020,Find out how you can protect yourself and love...,Extremely Positive,find out how you can protect yourself and love...
3,4,44956,Chicagoland,02-03-2020,#Panic buying hits #NewYork City as anxious sh...,Negative,# panic buying hits # newyork city as anxious ...
4,5,44957,"Melbourne, Victoria",03-03-2020,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral,# toiletpaper # dunnypaper # coronavirus # cor...


In [ ]:
import nltk
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("punkt")
nltk.download("punkt_tab")

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string

stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

def advanced_clean(text):
    text = text.lower()
    words = nltk.word_tokenize(text)
    words = [w for w in words if w not in stop_words and w not in string.punctuation]
    words = [lemmatizer.lemmatize(w) for w in words]
    return " ".join(words)

df["clean_text_final"] = df["clean_text_hf"].apply(advanced_clean)
df.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment,clean_text_hf,clean_text_final
0,1,44953,NYC,02-03-2020,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative,trending : new yorkers encounter empty superma...,trending new yorkers encounter empty supermark...
1,2,44954,"Seattle, WA",02-03-2020,When I couldn't find hand sanitizer at Fred Me...,Positive,when i couldn ' t find hand sanitizer at fred ...,find hand sanitizer fred meyer turned amazon 1...
2,3,44955,NaN,02-03-2020,Find out how you can protect yourself and love...,Extremely Positive,find out how you can protect yourself and love...,find protect loved one coronavirus
3,4,44956,Chicagoland,02-03-2020,#Panic buying hits #NewYork City as anxious sh...,Negative,# panic buying hits # newyork city as anxious ...,panic buying hit newyork city anxious shopper ...
4,5,44957,"Melbourne, Victoria",03-03-2020,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral,# toiletpaper # dunnypaper # coronavirus # cor...,toiletpaper dunnypaper coronavirus coronavirus...


# ***Sentiment Analysis Using Hugging Face***

In [ ]:
!pip install transformers torch


In [ ]:
from transformers import pipeline

sentiment_model = pipeline("sentiment-analysis")


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cpu


In [ ]:
def get_sentiment(text):
    result = sentiment_model(text[:512])[0]
    return result['label'], result['score']

df['sentiment_label'], df['sentiment_score'] = zip(*df['clean_text_final'].astype(str).apply(get_sentiment))
df.head()



,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment,clean_text_hf,clean_text_final,sentiment_label,sentiment_score
0,1,44953,NYC,02-03-2020,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative,trending : new yorkers encounter empty superma...,trending new yorkers encounter empty supermark...,NEGATIVE,0.999386
1,2,44954,"Seattle, WA",02-03-2020,When I couldn't find hand sanitizer at Fred Me...,Positive,when i couldn ' t find hand sanitizer at fred ...,find hand sanitizer fred meyer turned amazon 1...,NEGATIVE,0.997122
2,3,44955,NaN,02-03-2020,Find out how you can protect yourself and love...,Extremely Positive,find out how you can protect yourself and love...,find protect loved one coronavirus,POSITIVE,0.984344
3,4,44956,Chicagoland,02-03-2020,#Panic buying hits #NewYork City as anxious sh...,Negative,# panic buying hits # newyork city as anxious ...,panic buying hit newyork city anxious shopper ...,NEGATIVE,0.991502
4,5,44957,"Melbourne, Victoria",03-03-2020,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral,# toiletpaper # dunnypaper # coronavirus # cor...,toiletpaper dunnypaper coronavirus coronavirus...,NEGATIVE,0.997864


In [ ]:
df['sentiment_value'] = df['sentiment_label'].apply(lambda x: 1 if x == 'POSITIVE' else 0)


In [ ]:
print(df['sentiment_label'].value_counts())


sentiment_label
NEGATIVE    3574
POSITIVE     224
Name: count, dtype: int64


In [ ]:
print(df['sentiment_value'].mean())


0.05897840968931016


In [ ]:
df.to_csv("sentiment_output.csv", index=False)



In [ ]:
from google.colab import files
files.download("sentiment_output.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>